# Ontologien mit RDF Schema

In dieser Übung benutzen wir die in der Vorlesung vorgestellte Ontologie (Datei `solar-system-planet.ttl`) über Planeten des Sonnensystems und erweitern diese mit Protégé. Zudem verwenden wir die Ontologie in SPARQL Abfragen. Zuerst laden wir benötigte Python Module (insb. wenn Sie Azure verwenden) und Hilfsfunktionen.

In [2]:
#!pip install rdflib

In [3]:
import pandas as pd
from io import BytesIO
from io import StringIO
from rdflib import Graph
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer
from IPython.display import display

pd.set_option('display.max_colwidth', 200)

g1 = Graph()
r = g1.parse('solar-system-planet.ttl', format='turtle')

def query(g, q):
    serializer = CSVResultSerializer(g.query(q))
    output = BytesIO()
    serializer.serialize(output)
    display(pd.read_csv(StringIO(output.getvalue().decode())))

Schauen wir mal welche Planeten (Instanzen) unsere Ontologie bereits kennt.

In [4]:
query(g1, """
SELECT ?planet 
WHERE {
    ?p rdf:type [ rdfs:label "Terrestrial Planet"@en ] . 
    ?p rdfs:label ?planet
}
""")

,planet
0,Earth
1,Mars


Die Klasse "Terrestrial Planet" ist eine Unterklasse welcher Klasse? Schauen wir mal nach.

In [5]:
query(g1, """
SELECT ?class 
WHERE {
    ?c rdfs:label "Terrestrial Planet"@en . 
    ?c rdfs:subClassOf [ rdfs:label ?class ]
}
""")

,class
0,Solar System Planet


Schauen wir uns nun noch an, was die Ontologie über den Planeten Erde kennt.

In [6]:
query(g1, """
SELECT ?property ?object
WHERE {
    ?e rdfs:label "Earth"@en . 
    ?p rdfs:label ?property .
    { ?e ?p ?object }
    UNION
    { ?e ?p [ rdfs:label ?object ] }
    FILTER (isLiteral(?object))
}
""")

,property,object
0,has equatorial radius,6371
1,has satellite,Moon


Gehen Sie nun auf [http://webprotege.stanford.edu](http://webprotege.stanford.edu) und melden Sie sich an. Erstellen Sie nach erfolgreichem Login ein neues Projekt. Nennen Sie das Projekt `solar-system-planet-extended` und laden Sie die `solar-system-planet.ttl` Datei im Verzeichnis `notebooks/lecture-12` rauf. Die Maske sollte wie folgt aussehen.

![](webprotege-create-project.png)


Wählen Sie dann das neu erstellte Projekt. Schauen Sie sich die Ontologie etwas an. Klappen Sie die Klasse `Solar System Planet` auf. Welche Unterklassen gibt es? Wählen Sie dann das Tab `Properties` und schauen Sie sich die object property `has satellite` genauer an. Was sind die `Domain` und `Range`?

In [8]:
# Welche Unterklassen von Solar System Planet gibt es? Antwort: Terrestrial Planet und Gas Giant

# Was ist die Domain der has satellite property? Antwort: Solar System Planet

# Was ist die Range der has satellite property? Antwort: Solar System Satellite

Im Tab `Properties` wählen Sie nun die data properties und schauen Sie sich die `has physical characteristic` property an.

In [9]:
# Welche sub properties hat die has physical characteristic? Antwort: has equatorial radius und has mass

Wählen Sie nun das Tab `Individuals` und die Klasse `Terrestrial Planet`. Wie erwartet, sind hier die Instanzen `Earth` und `Mars` zu finden. Wählen Sie nun `Earth` und schauen Sie sich die Relationen (`Relationships`) an. Auch hier erwarten wir anhand der SPARQL Resultate oben Information über den Satellit und Radius.

Erweitern Sie die Ontologie nun mit den folgendem Wissen. Setzen Sie bei neuen Relationen und Instanzen das Language Tag des labels jeweils auf "en" für Englisch. Verwenden Sie Werte (Daten) aus Wikipedia:

* Fügen Sie "has surface area" als weitere (physical characteristic) data property hinzu
* Fügen Sie Venus als terrestrischer Planet hinzu
* Fügen Sie die Fläche von Venus, Mars und Erde hinzu
* Fügen Sie die beiden Mars Satelliten hinzu
* Fügen Sie Jupiter und Saturn als Gas Giganten hinzu

Sind Sie zufrieden, schliessen Sie das Projekt und auf der Hauptseite wählen Sie dann für das Projekt im Menu ganz rechts `Download`. Wählen Sie `Turtle` als Format. Sie erhalten eine ZIP Datei. Diese enthält die Datei `root-ontology.ttl` in einem Unterverzeichnis. Nennen Sie diese Datei auf `solar-system-planet-extended.ttl` um und laden Sie die Datei dann in Ihr lecture-12 Verzeichnis. 

Nun schauen wir, ob sich bei der erweiterten Ontologie was geändert hat und zwar so wie Sie es erwarten.

In [10]:
g2 = Graph()
r = g2.parse('solar-system-planet-extended.ttl', format='turtle')

In [11]:
query(g2, """
SELECT ?planet 
WHERE {
    ?p rdf:type [ rdfs:label "Terrestrial Planet"@en ] . 
    ?p rdfs:label ?planet
}
""")

,planet
0,Earth
1,Venus
2,Mars


In [12]:
query(g2, """
SELECT ?planet ?property ?object
WHERE {
    ?e rdfs:label ?planet . 
    ?p rdfs:label ?property .
    { ?e ?p ?object }
    UNION
    { ?e ?p [ rdfs:label ?object ] }
    FILTER (isLiteral(?object))
}
ORDER BY (?planet)
""")

,planet,property,object
0,Earth,has surface area,510072000
1,Earth,has satellite,Moon
2,Earth,has equatorial radius,6371
3,Mars,has surface area,144798500
4,Mars,has satellite,Phobos
5,Mars,has satellite,Deimos
6,Venus,has surface area,460230000
